# Cuaderno Clase PLN - GloVe y FastText

**Objetivos:**
*   Conocer enfoques alternativos a Word2Vec: GloVe (conteos globales) y FastText (subpalabras).
*   Entender la ventaja clave de FastText para manejar palabras fuera de vocabulario (OOV - Out Of Vocabulary).
*   Cargar y usar vectores FastText pre-entrenados.
*   Comparar resultados y capacidades (especialmente OOV) con Word2Vec.
*   Reflexionar sobre cómo evaluar la calidad de los embeddings y detectar sesgos.

**Agenda:**
1.  Instalaciones e Importaciones
2.  Repaso Rápido: Word2Vec y sus limitaciones (OOV)
3.  GloVe: Vectores Globales desde Co-ocurrencias
4.  FastText: El Poder de las Subpalabras (¡Adiós OOV!)
5.  Cargando Vectores FastText Pre-entrenados
6.  Explorando FastText: Similitud, Analogías y ¡OOV!
7.  Comparativa: Word2Vec vs FastText (foco en OOV)
8.  Micro-Laboratorio (Ejercicio Práctico)
9.  Brainstorming: Evaluación y Detección de Sesgos

# 1. Instalaciones e Importaciones

In [ ]:
# Necesitamos gensim principalmente
!pip install gensim > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip uninstall gensim -y # Remove the existing gensim installation
!pip install gensim # Reinstall gensim to align with the NumPy version
# Restart the kernel to ensure the changes take effect

Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.6 MB)


In [ ]:
import gensim
from gensim.models import KeyedVectors, FastText # Ahora importamos FastText también
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas.")

# **Importante:** Para este cuaderno, idealmente necesitaremos:
# 1. El modelo Word2Vec cargado previamente (para comparar).
# 2. Un modelo FastText pre-entrenado para español (¡necesitamos descargarlo!).

Librerías importadas.


# 2. Repaso Rápido: Word2Vec y sus limitaciones (OOV)

Recordemos:
*   Word2Vec (CBOW/Skip-gram) aprende embeddings prediciendo palabras en contextos locales.
*   Genera vectores densos que capturan semántica (similitud, analogías).
*   Usamos modelos pre-entrenados porque entrenarlos es costoso.

**Una limitación importante:** Word2Vec asigna un vector a cada palabra *completa* que vio durante el entrenamiento. Si en tiempo de uso aparece una palabra que **NO estaba en el vocabulario original** (Out-Of-Vocabulary - OOV):
*   Un error tipográfico ("computadorra").
*   Una palabra nueva o muy rara ("covid", "guasap").
*   Una variante morfológica no vista ("cantábamos").

**¿Qué pasa con Word2Vec?** ¡Generalmente da un error (`KeyError`) o asigna un vector genérico de "desconocido" (UNK - Unknown), perdiendo toda información específica! Esto es un problema en aplicaciones reales.

# 3. GloVe: Vectores Globales desde Co-ocurrencias

GloVe (Global Vectors for Word Representation) de Stanford es otra técnica popular para obtener embeddings.

*   **Enfoque Diferente:** En lugar de predecir contextos locales (como Word2Vec), GloVe se enfoca en las **estadísticas globales de co-ocurrencia** de palabras en todo el corpus.
*   **Idea Central:** La *relación* entre las probabilidades de co-ocurrencia de dos palabras con una tercera palabra "sonda" puede revelar información sobre su significado. Por ejemplo, la probabilidad de que "hielo" aparezca cerca de "sólido" será alta, mientras que cerca de "gas" será baja. La probabilidad de que "vapor" aparezca cerca de "gas" será alta y cerca de "sólido" baja. GloVe modela estas *proporciones* de probabilidades.
*   **Entrenamiento:** Construye una matriz gigante de co-ocurrencia (cuántas veces la palabra X aparece cerca de la palabra Y) y luego usa factorización de matrices para encontrar los vectores de palabras que mejor explican esa matriz.
*   **Resultado:** Embeddings densos similares a Word2Vec, a menudo muy buenos para tareas de similitud y analogía.
*   **Limitación OOV:** Al igual que Word2Vec, GloVe tradicionalmente opera a nivel de palabra completa, por lo que también sufre del problema de OOV.

*(Nota: Encontrar modelos GloVe pre-entrenados y de buena calidad para español puede ser más difícil que para Word2Vec o FastText. A menudo se distribuyen en formato texto).*

# 4. FastText: El Poder de las Subpalabras (¡Adiós OOV!)

FastText, desarrollado por Facebook AI Research (FAIR), es una extensión inteligente de Word2Vec (Skip-gram) que aborda directamente el problema OOV.

*   **¡La Gran Idea!:** FastText no trata las palabras como unidades indivisibles. Representa cada palabra como una **bolsa de n-gramas de caracteres**, además de la palabra completa.
    *   Ejemplo: Para la palabra "amarillo" y usando n=3 (trigramas):
        *   Se consideran n-gramas como: `<am`, `ama`, `mar`, `ari`, `ril`, `ill`, `llo`, `lo>` (con `<` y `>` marcando inicio y fin).
        *   También se considera la palabra completa: `<amarillo>`
*   **Aprendizaje:** El modelo aprende vectores no solo para las palabras completas, sino **para todos los n-gramas de caracteres** vistos en el corpus.
*   **Vector Final:** El vector de una palabra se obtiene **sumando los vectores de todos sus n-gramas de caracteres** (y el vector de la palabra completa si existe).

**¡La Ventaja Clave - Manejo de OOV!**
*   Si aparece una palabra nueva que no estaba en el vocabulario (ej: "amarillento"), FastText **aún puede construir un vector para ella**. ¿Cómo? Sumando los vectores de los n-gramas que sí conoce y que forman parte de la palabra nueva (ej: `<am`, `ama`, `mar`, `ari`, `ril`, `ill`, `lle`, `len`, `ent`, `nto`, `to>`).
*   El vector resultante captura información de las "partes" de la palabra, lo que a menudo resulta en una representación razonable incluso para palabras desconocidas.

**Otros Beneficios:**
*   Funciona muy bien para **lenguajes morfológicamente ricos** (como el español) donde muchas palabras comparten raíces y afijos (ej: "nación", "nacional", "nacionalidad"). Comparten n-gramas, sus vectores estarán relacionados.
*   Es más robusto a **errores tipográficos** ("computadorra" compartirá muchos n-gramas con "computadora").

# 5. Cargando Vectores FastText Pre-entrenados

Al igual que con Word2Vec, lo usual es usar modelos FastText pre-entrenados.

**Tarea:** Descargar los vectores pre-entrenados para **español** desde el sitio oficial de FastText: [https://fasttext.cc/docs/en/pretrained-vectors.html](https://fasttext.cc/docs/en/pretrained-vectors.html)

*   **Importante:** Descargar el archivo `.bin`. Este formato contiene no solo los vectores de las palabras del vocabulario, sino también la información de los n-gramas necesaria para calcular vectores de palabras OOV. El formato `.vec` es más pequeño pero pierde esta capacidad OOV.
*   El archivo `.bin` para español también es grande (varios GB). Anoten la ruta donde lo guardan.

**¡Reemplacen `'RUTA_AL_ARCHIVO_FASTTEXT.bin'` con la ruta real!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- ¡¡¡ MODIFICAR ESTA RUTA !!! ---
# Ejemplo de ruta (si lo subieron a Colab o está en el mismo directorio):
# path_to_fasttext = 'cc.es.300.bin' # Nombre común del archivo FastText para español
# Ejemplo de ruta (si está en Google Drive montado):
path_to_fasttext = '/content/drive/MyDrive/Clases/HABLA/004/PRA/vectors/wiki.es.bin' # Ajustar según su estructura

In [ ]:
# Intentar cargar el modelo FastText
try:
    print("Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)")
    # Usamos FastText.load_fasttext_format para cargar modelos .bin de FastText
    fasttext_model = gensim.models.fasttext.load_facebook_model(path_to_fasttext)
    # Los vectores están dentro del atributo .wv (como en Word2Vec cargado con KeyedVectors)
    fasttext_vectors = fasttext_model.wv
    print(f"¡Vectores FastText cargados! Vocabulario (estimado): {len(fasttext_vectors.index_to_key)} palabras. Dimensión: {fasttext_vectors.vector_size}")
    # Nota: El tamaño del vocabulario explícito puede ser menor en .bin, pero puede generar para OOV.
except FileNotFoundError:
    print(f"Error: No se encontró el archivo FastText en la ruta '{path_to_fasttext}'.")
    print("Por favor, descarga el archivo .bin pre-entrenado para español desde el sitio de FastText")
    print("y asegúrate de que la variable 'path_to_fasttext' tenga la ruta correcta.")
    fasttext_vectors = None
except Exception as e:
    print(f"Ocurrió un error al cargar el modelo FastText: {e}")
    fasttext_vectors = None

Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)
¡Vectores FastText cargados! Vocabulario (estimado): 985667 palabras. Dimensión: 300


In [ ]:
# Intentemos recargar el modelo Word2Vec para comparar
# --- ¡¡¡ MODIFICAR ESTA RUTA TAMBIÉN SI ES NECESARIO !!! ---
path_to_word2vec = '/content/drive/MyDrive/Clases/HABLA/004/PRA/vectors/SBW-vectors-300-min5.bin.gz' # La ruta del martes
word2vec_vectors = None

In [ ]:
try:
    print("\nRecargando vectores Word2Vec...")
    word2vec_vectors = KeyedVectors.load_word2vec_format(path_to_word2vec, binary=True)
    print("Vectores Word2Vec recargados.")
except Exception as e:
    print(f"No se pudo recargar Word2Vec desde '{path_to_word2vec}': {e}")


Recargando vectores Word2Vec del martes...
Vectores Word2Vec recargados.


# 6. Explorando FastText: Similitud, Analogías y ¡OOV!

Si `fasttext_vectors` se cargó, podemos hacer pruebas similares a las de Word2Vec, pero prestando especial atención a las palabras OOV.

In [ ]:
if fasttext_vectors:
    # --- Pruebas estándar (similitud, analogía) ---
    print("\n--- Explorando FastText ---")
    # Similitud
    try:
        similares_rey_ft = fasttext_vectors.most_similar('rey', topn=5)
        print("FastText - Más similares a 'rey':", similares_rey_ft)
    except KeyError:
        print("FastText - Palabra 'rey' no encontrada.") # Raro si el modelo es bueno

    # Analogía
    try:
        analogia_reina_ft = fasttext_vectors.most_similar(positive=['rey', 'mujer'], negative=['hombre'], topn=1)
        print("FastText - rey - hombre + mujer ≈", analogia_reina_ft)
    except KeyError as e:
        print(f"FastText - Error en analogía rey/reina: falta la palabra '{e.args[0]}'")


    # --- ¡La prueba OOV! ---
    print("\n--- Probando Palabras Fuera de Vocabulario (OOV) ---")
    palabras_oov = [
        "computadorra", # Error tipográfico
        "wasapear",     # Neologismo / Palabra coloquial
        "covid",        # Palabra relativamente nueva (depende del corpus)
        "programadorazo", # Palabra inventada con sufijo común
        "desayunábamos", # Forma verbal menos frecuente
        "internauta"    # Palabra estándar, veamos si está
    ]

    for palabra in palabras_oov:
        print(f"\nPalabra OOV: '{palabra}'")
        # ¿Está explícitamente en el vocabulario? (Puede que sí si el corpus era reciente)
        in_vocab = palabra in fasttext_vectors
        print(f"  ¿En vocabulario explícito? {in_vocab}")

        # Intentar obtener el vector (¡FastText debería poder!)
        try:
            vector_oov = fasttext_vectors[palabra]
            print(f"  ¡Vector obtenido! (Dim: {vector_oov.shape})")

            # Encontrar similares basados en el vector generado
            similares_oov = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"  Similares (FastText): {similares_oov}")
        except Exception as e:
            # Este error no debería ocurrir con un modelo .bin cargado correctamente
            print(f"  Error inesperado obteniendo vector/similares para '{palabra}': {e}")

else:
    print("\nNo se pudieron cargar los vectores FastText. Saltando la exploración.")


--- Explorando FastText ---
FastText - Más similares a 'rey': [('monarca', 0.7459232807159424), ('rey―', 0.6321003437042236), ('exmonarca', 0.6310685873031616), ('rey,', 0.6303314566612244), ('reina', 0.6260297298431396)]
FastText - rey - hombre + mujer ≈ [('reina', 0.6612293720245361)]

--- Probando Palabras Fuera de Vocabulario (OOV) ---

Palabra OOV: 'computadorra'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('computador', 0.8841244578361511), ('computadora/ordenador', 0.8638773560523987), ('computadora,', 0.8576205372810364)]

Palabra OOV: 'wasapear'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('dissapear', 0.6578547358512878), ('tapear', 0.6350954174995422), ('papear', 0.5647345781326294)]

Palabra OOV: 'covid'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('covides', 0.7112458944320679), ('coview', 0.6201539635658264), ('covida', 0.5898

# 7. Comparativa: Word2Vec vs FastText (foco en OOV)

Ahora, si ambos modelos (`word2vec_vectors` y `fasttext_vectors`) están cargados, podemos comparar directamente su comportamiento con palabras OOV.

In [ ]:
# --- Comparación directa OOV ---
if word2vec_vectors and fasttext_vectors:
    print("\n--- Comparación OOV: Word2Vec vs FastText ---")

    # Reutilizamos la lista de palabras OOV
    palabras_oov = [
        "computadorra", "wasapear", "covid", "programadorazo", "desayunábamos", "internauta"
    ]

    for palabra in palabras_oov:
        print(f"\n--- Palabra: '{palabra}' ---")

        # Intentar con Word2Vec
        print("  Intentando con Word2Vec:")
        try:
            vector_w2v = word2vec_vectors[palabra]
            similares_w2v = word2vec_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Encontrada! Vector: {vector_w2v.shape}, Similares: {similares_w2v}")
        except KeyError:
            print("    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).")
        except Exception as e:
            print(f"    ERROR inesperado con Word2Vec: {e}")

        # Intentar con FastText
        print("  Intentando con FastText:")
        try:
            vector_ft = fasttext_vectors[palabra]
            similares_ft = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Vector generado! Vector: {vector_ft.shape}, Similares: {similares_ft}")
        except Exception as e:
            # No debería fallar por KeyError, pero podría haber otro error
            print(f"    ERROR inesperado con FastText: {e}")

else:
    print("\nNo se pudieron cargar ambos modelos (Word2Vec y FastText) para comparar.")


--- Comparación OOV: Word2Vec vs FastText ---

--- Palabra: 'computadorra' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('computador', 0.8841244578361511), ('computadora/ordenador', 0.8638773560523987), ('computadora,', 0.8576205372810364)]

--- Palabra: 'wasapear' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('dissapear', 0.6578547358512878), ('tapear', 0.6350954174995422), ('papear', 0.5647345781326294)]

--- Palabra: 'covid' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('covides', 0.7112458944320679), ('coview', 0.6201539635658264), ('covida', 0.5898890495300293)]

--- Palabra: 'progr

**Conclusión de la Comparativa:**

Deberían observar que para la mayoría (o todas) las palabras OOV, Word2Vec lanza un `KeyError`, mientras que FastText logra generar un vector y encontrar palabras similares (que pueden o no ser muy coherentes, dependiendo de qué tan "rara" sea la palabra OOV, pero *intenta* algo basado en sus partes).

**¿Cuándo usar FastText?**
*   Cuando trabajas con texto ruidoso (redes sociales, OCR) con typos.
*   Cuando tratas con lenguajes morfológicamente ricos (español, alemán, etc.).
*   Cuando esperas encontrar palabras nuevas o raras en tus datos de aplicación.
*   **En general, para español, FastText suele ser una opción más robusta y recomendable que Word2Vec o GloVe.**

**Desventaja:** Los modelos `.bin` de FastText son más grandes en disco y consumen más RAM porque almacenan información de los n-gramas.

# 8. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:** (Asumiendo que `fasttext_vectors` y `word2vec_vectors` están cargados)

1.  **Comparación de Resultados:**
    *   Elegir 3 palabras que **sí** estén en ambos vocabularios (ej: 'gato', 'correr', 'inteligencia').
    *   Para cada palabra, obtener las 5 más similares usando `word2vec_vectors.most_similar()` y `fasttext_vectors.most_similar()`.
    *   Comparar las listas de similares. ¿Son idénticas? ¿Muy parecidas? ¿Diferentes? ¿Cuál les parece "mejor" o más coherente? Anotar observaciones.

2.  **Test OOV Exhaustivo:**
    *   Crear una lista propia de 10 palabras OOV. Incluyan:
        *   Errores tipográficos comunes (ej: "hobmre", "qeu", "dicimbre").
        *   Diminutivos/Aumentativos (ej: "perrito", "casita", "libraco").
        *   Formas verbales conjugadas (ej: "habíamos comido", "cantasteis").
        *   Palabras inventadas pero plausibles (ej: "tecnoestrés", "computofilia").
    *   Para **cada** palabra OOV de su lista:
        *   Verificar si da `KeyError` en `word2vec_vectors`.
        *   Obtener las 3 palabras más similares usando `fasttext_vectors`. Anotar los resultados. ¿Los similares que da FastText tienen algún sentido basado en las partes de la palabra OOV?

3.  **Discusión:**
    *   ¿En qué tipo de aplicación real (ej: un chatbot de atención al cliente, un sistema de recomendación de noticias, un corrector ortográfico) creen que la capacidad OOV de FastText marcaría una diferencia significativa respecto a usar Word2Vec? ¿Por qué?

# 9. Brainstorming: Evaluación y Detección de Sesgos

Hemos visto diferentes formas de crear embeddings (Word2Vec, GloVe, FastText) y cómo usarlos. Pero, ¿cómo sabemos si un conjunto de embeddings es "bueno"? ¿Y cómo abordamos el problema de los sesgos que mencionamos en clases anteriores?

**Pregunta:** **¿Cómo podemos evaluar la calidad de los word embeddings y detectar posibles sesgos?**

*   **Evaluación Intrínseca:**
    *   Medir qué tan bien funcionan los embeddings en tareas específicas relacionadas con las palabras mismas, **sin** una aplicación final.
    *   Ejemplos:
        *   **Similitud de Palabras:** Comparar la similitud coseno entre pares de palabras según los embeddings, con juicios de similitud dados por humanos (datasets estándar como WordSim-353, SimLex-999, adaptados o creados para español).
        *   **Tareas de Analogía:** Ver qué tan bien resuelven analogías (`rey - hombre + mujer = ?`). Hay datasets estándar de analogías (Google Analogies, BATS).
*   **Evaluación Extrínseca:**
    *   Usar los embeddings como **características de entrada (features)** para un modelo de PLN más complejo que resuelve una tarea final (downstream task).
    *   Ejemplos: Clasificación de sentimientos, reconocimiento de entidades nombradas, clasificación de temas.
    *   Medir si usar estos embeddings mejora el rendimiento (precisión, F1-score, etc.) del sistema final comparado con no usarlos o usar otros embeddings. **Esta suele ser la evaluación más relevante en la práctica.**
*   **Detección de Sesgos:**
    *   **Analogías Específicas:** Crear analogías diseñadas para revelar sesgos sociales (género-profesión, raza-sentimiento, etc.). `programador - hombre + mujer = ?`.
    *   **Pruebas de Asociación Implícita (como WEAT - Word Embedding Association Test):** Miden matemáticamente qué tan asociados están ciertos grupos de palabras (ej: nombres masculinos vs femeninos) con ciertos atributos (ej: carreras científicas vs artísticas, adjetivos positivos vs negativos) dentro del espacio de embeddings.
    *   **Visualización:** A veces, proyectar grupos específicos (hombres/mujeres, profesiones) a 2D puede revelar agrupaciones o direcciones sesgadas.
    *   **Auditoría de Vecinos Cercanos:** Ver los `most_similar` para palabras sensibles.

**Una vez detectado el sesgo, ¿qué hacemos?** ¿Existen técnicas para mitigarlo ("debiasing")? ¿Es mejor buscar/crear corpus menos sesgados? ¿O simplemente ser transparentes sobre los sesgos del modelo?

**(Discusión en grupo)**

# GUÍA DE ESTUDIO - FASTTEXT Y GLOVE

## Preguntas y Respuestas Clave

### **Innovaciones Arquitectónicas**

**P: ¿Cuál es la innovación principal de FastText sobre Word2Vec?**  
R: FastText representa palabras como suma de n-gramas de caracteres, no como unidades indivisibles. Esto permite generar vectores para palabras nunca vistas (OOV).

**P: ¿Cómo funciona FastText con palabras desconocidas?**  
R: Descompone la palabra en n-gramas ("amarillo" → "<am", "ama", "mar", etc.) y suma los vectores de los n-gramas conocidos para generar un vector para la palabra completa.

**P: ¿En qué se diferencia GloVe de Word2Vec conceptualmente?**  
R: Word2Vec predice contextos locales. GloVe usa estadísticas globales de co-ocurrencia de todo el corpus para factorizar una matriz de co-ocurrencias.

### **Ventajas Específicas**

**P: ¿Por qué FastText es especialmente bueno para español?**  
R: El español es morfológicamente rico (muchas conjugaciones, derivaciones). FastText captura estas relaciones morfológicas a través de n-gramas compartidos.

**P: ¿Qué ventajas tiene FastText para texto ruidoso?**  
R: Es robusto a errores tipográficos porque "computadorra" comparte muchos n-gramas con "computadora", generando un vector similar.

**P: ¿Cuándo elegir GloVe sobre Word2Vec?**  
R: Cuando necesitas mejor aprovechamiento de estadísticas corpus-wide o tienes computational budget limitado para múltiples pasadas sobre el corpus.

### **Implementación Práctica**

**P: ¿Por qué es importante descargar archivos .bin de FastText?**  
R: El formato .bin incluye información de n-gramas necesaria para manejar OOV. El formato .vec solo tiene vectores de palabras conocidas.

**P: ¿Qué indica que una palabra no está en vocabulario explícito pero FastText puede procesarla?**  
R: `palabra in fasttext_vectors` devuelve False, pero `fasttext_vectors[palabra]` aún funciona generando un vector dinámicamente.

### **Evaluación y Comparación**

**P: ¿Cómo evaluar si FastText produce mejores embeddings que Word2Vec?**  
R: Evaluación intrínseca (tareas de similitud, analogías) y extrínseca (performance en tareas downstream como clasificación de texto).

**P: ¿Qué trade-offs tiene FastText?**  
R: Pro: maneja OOV, mejor morfología. Contra: mayor tamaño de modelo, mayor uso de RAM, menor interpretabilidad de n-gramas.

### **Casos de Uso**

**P: ¿En qué aplicaciones sería crítica la capacidad OOV de FastText?**  
R: Redes sociales (jerga, typos), análisis de texto histórico (ortografía variable), corpus especializados (terminología técnica), lenguas con poca documentación.

**P: ¿Cómo manejaría lenguaje inclusivo con FastText?**  
R: "Estudiantx" se procesaría usando n-gramas de "estudiante" + sufijo, capturando la relación semántica automáticamente.

## Puntos Clave para Recordar

1. **FastText = Word2Vec + n-gramas de caracteres**
2. **OOV no es problema** con FastText (vs KeyError en Word2Vec)
3. **GloVe = estadísticas globales** vs predicción local
4. **Para español: FastText > Word2Vec** por morfología rica
5. **Trade-off: capacidad vs complejidad computacional**
6. **Archivos .bin cruciales** para funcionalidad OOV

## Consideraciones Importantes

- FastText puede generar vectores para cualquier string, incluso nonsense
- La calidad del vector OOV depende de qué tan informativos sean los n-gramas
- GloVe puede ser difícil de encontrar pre-entrenado para español
- Modelos más grandes requieren más RAM y tiempo de carga
- No todos los n-gramas son igualmente informativos

## Conexión con Próxima Clase

Ahora conocen múltiples arquitecturas de embeddings. La próxima clase integrará todos estos métodos en **pipelines prácticos**, comparando su rendimiento en tareas reales y decidiendo cuándo usar cada uno.

---
*Consejo: Testa FastText con argentinismos o errores de tipeo comunes en WhatsApp. ¿Genera vectores sensatos para "kjjjj" o "messirve"?*